# Manhattan Rental Apartments Clustering
## - Clustering

### Xian Lai
xlai4@fordham.edu   
Apr.2017

=======================================================

In the last jupyter notebook, we select the final clustering model using combined score with hyperparameters:  
**{'IDWpower':0.1, 'n_ngb':13, 'method':'complete', 'metric':'euclidean'}**

Next we will use this model on the full building dataset and query information we are interested in.

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import sys; sys.path.append('modules')
import HierarchicalClustering as hc
import Visual_static as vis
from bokeh.io import output_notebook, push_notebook, show
from bokeh.layouts import row
output_notebook()

Loading BokehJS ...

In [ ]:
df_listing = pd\
    .read_pickle('data/market_set_cleaned.pickle')\
    .reset_index(drop=True)
df_building = pd\
    .read_pickle('data/building_set_cleaned.pickle')\
    .drop(labels=['DOITT_ID', 'footprint'], axis=1)\
    .reset_index(drop=True)
    
HC = hc.HierarchicalClustering(df_listing, df_building)
HC.prepareData(IDWpower=0.1, n_ngb=13)
HC.clustering(
    n_clusters=600, method='complete', metric='euclidean'
)

Interpolation done.


## 1. Querying clustering using cluster statistics

In the process of clustering, we calculate some statistics for each cluster:
- Popularity mean
- Popularity variance
- cluster size
- cluster area

What we can do with these statistics are:
1. We can use them to filter clusters. For example, we can filter out 100 clusters with highest popularities.
2. We can visualize these clusters using statistics as color coding. For example, we can plot the clusters colored by their popularity mean.   
(green->red : unpopular->popular).

In [ ]:
# The dataset of each data point with their corresponding 
# cluster's statistics
HC.clusters.head() 

In [ ]:
v = vis.Visual(figWidth=400)
# v.plotClustering(HC.clusters, color='mean')

<img src="images/all.png" width="400">

### Examples:
In the 1st example, let's filter out the top 200 largest clusters in terms of area and plot them colored by mean, variance and size respectively. 

To do this, we will first sort the stats by the area column and take the top 200 clusters. And then we filter the dataset with the cluster indices corresponding to the top 200 clusters. At last we plot the clustering colored by different statistics.

In [ ]:
largeClusters_200 = HC.clusterStats\
    .copy()\
    .sort_values('area', ascending=False)\
    .head(n=200)
data = HC.clusters.copy()
mask = data['cluster'].isin(largeClusters_200.index)
data = data[mask]

In [ ]:
v1 = vis.Visual(figWidth=300, title='colored by mean')
v1.plotClustering(data, color='mean', show=False)
v2 = vis.Visual(figWidth=300, title='colored by variance', 
     x_range=v1.p.x_range, y_range=v1.p.y_range)
v2.plotClustering(data, color='variance', show=False)
v3 = vis.Visual(figWidth=300, title='colored by size', 
     x_range=v1.p.x_range, y_range=v1.p.y_range)
v3.plotClustering(data, color='size', show=False)
# show(row(v1.p, v2.p, v3.p))

<img src="images/top_area_clusters.png" width="1300">

In the 2nd example, we will investigate the clusters with large popularity variance and look at their popularity mean.

In [ ]:
largeVarClusters = HC.clusterStats.copy()\
    .sort_values('variance', ascending=False)\
    .head(n=200)
data = HC.clusters.copy()
data = data[data['cluster'].isin(largeVarClusters.index)]

In [ ]:
v1 = vis.Visual(figWidth=300, title="Full clustering")
v1.plotClustering(HC.clusters, color='mean', show=False)
v2 = vis.Visual(figWidth=300, title="Large variance clusters colored by mean")
v2.plotClustering(data, color='mean', bins=v1.bins, show=False)
v3 = vis.Visual(figWidth=300, title="Large variance clusters colored by variance")
v3.plotClustering(data, color='variance', show=False)
# show(row(v1.p, v2.p, v3.p))

<img src="images/top_variance_clusters.png" width="1300">

## 2. Query clustering using different building properties

Besides querying one clustering with its stats, we can also comparing different clusterings. Recall that we have different features in the listing dataset like price level, whether there is broker fee, whether there is fitness center in the building etc. We can use these features to create different listing datasets and then compare the clustering results coming from different datasets interpolated from different listing datasets.

As examples, we will compare the following clusters:
    - low price rentals vs high price rentals
    - 1-bedroom rentals, 2-bedroom rentals vs 4-bedroom rentals
    - rentals without elevators vs rentals with elevators
    - rentals without gym vs rentals with gym

In [ ]:
df_listing.head()

### Price levels:
The listing apartments are in 4 price bins. We can compare the clusterings interpolated from low, median and high price apartments.

In [ ]:
df_listing['price_bin'].describe()

In [ ]:
listing_datasets = [
    df_listing[df_listing['price_bin']==0],
    df_listing[df_listing['price_bin']==1],
    df_listing[df_listing['price_bin']==3]
]
print("The size of low price apartment listing dataset: %d" %len(listing_datasets[0]))
print("The size of median price apartment listing dataset: %d" %len(listing_datasets[1]))
print("The size of high price apartment listing dataset: %d" %len(listing_datasets[2]))

In [ ]:
HCs_price = []
for listing_dataset in listing_datasets:
    HC = hc.HierarchicalClustering(listing_dataset, df_building)
    HC.prepareData(IDWpower=0.1, n_ngb=13)
    HC.clustering(
        n_clusters=600, method='complete', metric='euclidean'
    )
    HCs_price.append(HC)

In [ ]:
datas = [HC.clusters for HC in HCs_price]
titles=[
    'low price rentals clustering', 
    'median price rentals clustering', 
    'high price rentals clustering'
]
v1 = vis.Visual(figWidth=300, title=titles[0])
v1.plotClustering(datas[0], show=False)
v2 = vis.Visual(figWidth=300, title=titles[1])
v2.plotClustering(datas[1], show=False)
v3 = vis.Visual(figWidth=300, title=titles[2])
v3.plotClustering(datas[2], show=False)
# show(row(v1.p, v2.p, v3.p))

<img src="images/prices.png" width="1300">

### Number of bedrooms:

In [ ]:
df_listing['bed'].describe()

In [ ]:
listing_datasets = [
    df_listing[df_listing['bed']==1],
    df_listing[df_listing['bed']==2],
    df_listing[df_listing['bed']==4]
]
print("The size of 1-bedroom apartment listing dataset: %d" %len(listing_datasets[0]))
print("The size of 2-bedroom apartment listing dataset: %d" %len(listing_datasets[1]))
print("The size of 4-bedroom apartment listing dataset: %d" %len(listing_datasets[2]))

In [ ]:
HCs_br = []
for listing_dataset in listing_datasets:
    HC = hc.HierarchicalClustering(listing_dataset, df_building)
    HC.prepareData(IDWpower=0.1, n_ngb=13)
    HC.clustering(
        n_clusters=600, method='complete', metric='euclidean'
    )
    HCs_br.append(HC)

In [ ]:
datas = [HC.clusters for HC in HCs_br]
titles=[
    '1-bedroom rentals clustering', 
    '2-bedroom rentals clustering', 
    '4-bedroom rentals clustering'
]
v1 = vis.Visual(figWidth=300, title=titles[0])
v1.plotClustering(datas[0], show=False)
v2 = vis.Visual(figWidth=300, title=titles[1])
v2.plotClustering(datas[1], show=False)
v3 = vis.Visual(figWidth=300, title=titles[2])
v3.plotClustering(datas[2], show=False)
# show(row(v1.p, v2.p, v3.p))

<img src="images/bedrooms.png" width="1300">

### Elevators:

In [ ]:
listing_datasets = [
    df_listing[df_listing['elevator']==0],
    df_listing[df_listing['elevator']==1]
]
print("The size of non-elevator apartment listing dataset: %d" %len(listing_datasets[0]))
print("The size of elevator apartment listing dataset: %d" %len(listing_datasets[1]))

In [ ]:
HCs_elev = []
for listing_dataset in listing_datasets:
    HC = hc.HierarchicalClustering(listing_dataset, df_building)
    HC.prepareData(IDWpower=0.1, n_ngb=13)
    HC.clustering(
        n_clusters=600, method='complete', metric='euclidean'
    )
    HCs_elev.append(HC)

In [ ]:
datas = [HC.clusters for HC in HCs_elev]
titles=['non-elevator rentals clustering', 'elevator rentals clustering']
v1 = vis.Visual(figWidth=450, title=titles[0])
v1.plotClustering(datas[0], show=False)
v2 = vis.Visual(figWidth=450, title=titles[1])
v2.plotClustering(datas[1], show=False)
# show(row(v1.p, v2.p))

<img src="images/elevators.png" width="600">

### Gyms:

In [ ]:
listing_datasets = [
    df_listing[df_listing['fitness_center']==0],
    df_listing[df_listing['fitness_center']==1]
]
print("The size of non-fitness_center apartment listing dataset: %d" 
      %len(listing_datasets[0]))
print("The size of fitness_center apartment listing dataset: %d" 
      %len(listing_datasets[1]))

In [ ]:
HCs_fc = []
for listing_dataset in listing_datasets:
    HC = hc.HierarchicalClustering(listing_dataset, df_building)
    HC.prepareData(IDWpower=0.1, n_ngb=13)
    HC.clustering(
        n_clusters=600, method='complete', metric='euclidean'
    )
    HCs_fc.append(HC)

In [ ]:
datas_fc = [HC.clusters for HC in HCs_fc]
titles=[
    'non-fitness_center rentals clustering', 
    'fitness_center rentals clustering'
]
v_fc_0 = vis.Visual(figWidth=450, title=titles[0])
v_fc_0.plotClustering(datas_fc[0], show=False)
v_fc_1 = vis.Visual(figWidth=450, title=titles[1])
v_fc_1.plotClustering(datas_fc[1], show=False)
# show(row(v_fc_0.p, v_fc_1.p))

<img src="images/gyms.png" width="600">


## Conclusion: 
By performing interpolation and clustering, we show that the geographic pattern do exist for Manhattan rental apartments. And it is affected differently by different apartment properties. 

Some apartment properties like price and number of bedrooms have heavy impacts on clustering. Some other properties like elevator or gym affect the building grouping in the same way but less compared to price or bedrooms.  They may be important, but they are independent of geographic location. 